In [2]:
from problog.program import PrologString
from problog import get_evaluatable
import os# define the arguments for the model:

def get_model_files(date:str, model_name:str):
    model_files = []
    for root, dirs, files in os.walk(os.path.join("src/generated/models", date)):
        for file in files:
            if file.startswith("prolog_model_"+model_name+"_"):
                model_files.append(os.path.join(root, file))

    models_string_dir = {}
    for file in model_files:
        with open(file, 'r') as f:
            models_string_dir[file] = f.read()
    print(f"Found {len(models_string_dir)}")
    return models_string_dir

def get_single_model_file_with_path(model_path:str):

    if os.path.isfile(model_path):
        with open(model_path, 'r') as f:
            model_string = f.read()
    else:
        print(f"this file could not be found")
    return model_string

def prolog_reasoning(model:str):
    result = get_evaluatable().create_from(PrologString(model)).evaluate()
    print("% ProbLog Inference Result：")
    for query_key, probability in result.items():
        print(f"{query_key} = {probability:.4f}")

In [10]:
model = """
zebra(Houses) :-
 houses(Houses),
 
member(house(englishman, _, _, red, _), Houses),
member(house(spaniard, dog, _, _, _), Houses),
member(house(_, _, coffee, green, _), Houses),
member(house(ukrainian, _, tea, _, _), Houses),
right_of(house(_, _, _, green, _), house(_, _, _, ivory, _), Houses),
member(house(_, snails, _, _, winston), Houses),
member(house(_, _, _, yellow, kools), Houses),
Houses = [_, _, house(_, _, milk, _, _), _, _],
Houses = [house(norwegian, _, _, _, _)|_],
next_to(house(_, _, _, _, chesterfields), house(_, fox, _, _, _), Houses),
next_to(house(_, _, _, _, kools), house(_, horse, _, _, _), Houses),
member(house(_, _, orange_juice, _, lucky_strikes), Houses),
member(house(japanese, _, _, _, parliaments), Houses),
next_to(house(norwegian, _, _, _, _), house(_, _, _, blue, _), Houses),
member(house(_, zebra, _, _, _), Houses),
member(house(_, _, water, _, _), Houses),
houses([
 house(_, _, _, _, _),
 house(_, _, _, _, _),
 house(_, _, _, _, _),
 house(_, _, _, _, _),
 house(_, _, _, _, _)
]).
right_of(A, B, [B, A | _]).
right_of(A, B, [_ | Y]) :- right_of(A, B, Y).
next_to(A, B, [A, B | _]).
next_to(A, B, [B, A | _]).
next_to(A, B, [_ | Y]) :- next_to(A, B, Y).
my_member(X, [X|_]).
my_member(X, [_|Y]) :- my_member(X, Y).
query(zebra(Houses)).
"""

prolog_reasoning(model)

UnknownClause: No clauses found for 'houses/1' at 3:2.

/Users/zhenzhili/miniforge3/envs/langda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
import numpy as np
import faiss
from typing import List
from config import paths
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.documents import BaseDocumentTransformer, Document

from langchain.document_loaders import PyMuPDFLoader

from langchain_huggingface import HuggingFaceEmbeddings

problog_official_doc_path = paths.get_absproj_path("problog-readthedocs-io-en-latest.pdf")
loader = PyMuPDFLoader(problog_official_doc_path)
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(documents)

# embedding_function = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
# Initialize Ollama embeddings using DeepSeek-R1
from concurrent.futures import ThreadPoolExecutor
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.embeddings import OllamaEmbeddings
embedding_function = OllamaEmbeddings(model="nomic-embed-text")

# # Parallelize embedding generation
# def generate_embedding(chunk:Document) -> List[float]:
#     return embedding_function.embed_query(chunk)

# with ThreadPoolExecutor() as executor:
#     embeddings = list(executor.map(generate_embedding, chunks))

#  Abstract structure for an index, supports adding vectors and searching them.
# All vectors provided at add or search time are 32-bit float arrays, although the internal representation may vary.
# index = faiss.IndexFlatL2(len(embedding_function.embed_query("hello world"))) # Input: vector dimension

vector_store = FAISS.from_documents(chunks, embedding_function)

vector_store.save_local(paths.get_absproj_path("faiss_langda_kb"))
# vector_store = FAISS(
#     embedding_function=embedding_function,
#     index=index,
#     docstore=InMemoryDocstore(),
#     index_to_docstore_id={},
# )

# from uuid import uuid4
# from langchain_core.documents import Document

# uuids = [str(uuid4()) for _ in range(len(chunks))]


In [ ]:
from config import paths
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.embeddings import OllamaEmbeddings

problog_official_doc_path = paths.get_absproj_path("problog-readthedocs-io-en-latest.pdf")
loader = PyMuPDFLoader(problog_official_doc_path)
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(documents)

embedding_function = OllamaEmbeddings(model="nomic-embed-text")

vector_store = FAISS.from_documents(chunks, embedding_function)

vector_store.save_local(paths.get_absproj_path("faiss_langda_kb"),index_name="problog_readthedocs")
